**Libraries Required**

In [ ]:
import os
import json
import pinecone
from langchain.llms import OpenAI
from langchain_openai import OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.pinecone import Pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import SemanticSimilarityExampleSelector, MaxMarginalRelevanceExampleSelector
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.memory import ConversationBufferMemory,ConversationEntityMemory
from langchain.chains.query_constructor.ir import Visitor
from langchain.chains import LLMChain
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)

In [ ]:
#Setting the environment variables
os.environ["OPENAI_API_KEY"] = ""
os.environ["PINECONE_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"]= ""
os.environ["LANGCHAIN_TRACING_V2"]= ""
os.environ["LANGCHAIN_PROJECT"]=""


#QueryProcessor class definition

class QueryProcessor:
    def __init__(self, prompt_file_path):
        self.prompt_file_path = prompt_file_path
        self.llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0.0)
        self.embeddings = OpenAIEmbeddings()
        self.output_parser = StrOutputParser()
        self.memory = ConversationEntityMemory(llm=self.llm)
      
        self.example_prompt = PromptTemplate(
                input_variables=["output"],
                template="Format: {output}",
            )

        self.example_selector = self.load_example_selector()
        
#function to load example selector
    def load_example_selector(self):
        examples = self.read_examples_from_file(self.prompt_file_path)
        return MaxMarginalRelevanceExampleSelector.from_examples(examples, self.embeddings, FAISS, k=1)

#Function to read examples from files 
    def read_examples_from_file(self, file_path):
        with open(file_path, 'r') as file:
            examples = json.load(file)
        return examples
    
#Function to process query
    def process_query(self, query, namespace, chat_history):
        #Creating pinecone vectorstore
        vectorstore = Pinecone.from_existing_index(index_name='langchainindex', embedding=self.embeddings, namespace=namespace)
        
        #Creating metadata field information
        metadata_field_info = [
            AttributeInfo(name="Wine Name", description="The name of the wine.", type="string"),
            AttributeInfo(name="Vintage Year", description="The year the wine was produced.", type="integer"),
            AttributeInfo(name="Region", description="The region where the wine originated.", type="string"),
            AttributeInfo(name="Grape Varietal", description="The grape variety used to make the wine.", type="string"),
            AttributeInfo(name="Vineyard", description="The vineyard or winery that produced the wine.", type="string"),
            AttributeInfo(name="Price", description="This is the price of the wine.", type="integer"),
            AttributeInfo(name="Pairs Well With or Best Matched", description="The wine that paired well with this.", type="string")
         ]
        document_content_description = "Brief summary of wines. It includes the wine name and its complete description."

        # Creating a prompt for the query constructor
        prompt = get_query_constructor_prompt(document_content_description,metadata_field_info)
        
        # Creating an output parser for structured query output
        output_parser = StructuredQueryOutputParser.from_components()
        # Constructing the query constructor with language model (llm) and output parser
        query_constructor = prompt | self.llm | output_parser
        # Creating a retriever object with the llm, vectorstore, and other parameters
        retriever = SelfQueryRetriever.from_llm(
            self.llm,
            vectorstore,
            document_content_description,
            metadata_field_info,
            enable_limit=True,
            search_type='mmr',
            verbose=True
        )

        # Printing the constructed filter
        print("Filter:", query_constructor.invoke(query))
        
        # Initializing a list to store fetched data             
        data_fetched_li = []
        # Getting relevant documents using the retriever
        data_fetched = retriever.get_relevant_documents(query)
        print("Retriever: ", data_fetched)
        # Extracting page content from fetched data
        for i in data_fetched:
            data_fetched_li.append(i.page_content)
        
        # Concatenating fetched data
        db_result = "".join(data_fetched_li)
        # Constructing a prompt for the language model
        prompt_sel = self.construct_prompt(query).split("\n\n")[0]
        # Generating response from the language model
        llm_result = self.generate_response(query, prompt_sel, db_result)
        # Extracting response text from llm_result
        response_dict = llm_result['text'].split(":")[-1].strip()
        # Appending the query and response to chat history
        chat_history.append({"Human:" :query, "AI:":response_dict})
        return response_dict

    def construct_prompt(self, query):
        # Creating a dynamic prompt template
        dynamic_prompt = FewShotPromptTemplate(
            example_selector=self.example_selector, # Selector for choosing few-shot example
            example_prompt=self.example_prompt,    # Prompt for few-shot example
            suffix="User: {user_query} ",          # Suffix added to the prompt, including the user's query
            input_variables=["user_query"],        # Input variables used in the prompt template

        )
        # Formatting the dynamic prompt with the user's query
        dynamic_prompt_str = str(dynamic_prompt.format(user_query=query)).replace("{","").replace("}","")
        # Returning the constructed prompt as a string
        return dynamic_prompt_str

    #Function to generate response based on prompt
    def generate_response(self,query, prompt_sel, db_result):
        # Creating a template for providing context to generate a response
        context_prompt = PromptTemplate(
            input_variables=["context"],   # Input variable for the context
            template="""Your job is to provide the relevant response.
                        You are provide a context, based on which you have to generate response:
                        
                        <context>
                        {context}
                        </context>
                        
                      
                       Your response should be brief, pertinent to the given context, and directly address the query. 
                       
                       Avoid generating unrelated responses and maintain coherence with the provided context."""
                    )
        # Formatting the context prompt with selected example and provided context
        example_selected_prompt = context_prompt.format(example=prompt_sel, context=db_result)
        # Creating the final prompt template for generating the response
        final_prompt = PromptTemplate(
            template=str(example_selected_prompt) +
                     """You are provided with information about the chat with the Human, if relevant.
                     User:{input}
                     Response:""",  # Including chat history and user input for generating response
            input_variables=['input'])  # Input variable for user input
        
        # Creating LLM chain with the final prompt
        llm_response_chain = LLMChain(llm=self.llm, prompt=final_prompt, memory=self.memory)
        # Generating response using the LLM chain with user input
        return llm_response_chain({'input': query})


# Example usage:
prompt_file_path = "..."  # Path to prompt file
query_processor = QueryProcessor(prompt_file_path)  # Initializing query processor

namespace= "wine" # Namespace for the query processing
chat_history = [] # Initializing chat history

# Continuously process user queries and provide responses
while True:
    query = str(input()) # Get user query
    response = query_processor.process_query(query=query, namespace=namespace, chat_history=chat_history) # Process query
    print("RESPONSE:", response)


 suggest me wines whose price is $10?


Filter: query=' ' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='Price', value=10) limit=None
Retriever:  [Document(page_content='Tina Fuerza Pinot Noir review titled "Great tasting !" says: "Looking for a great but budget friendly pinot noir? Look no further! Tina Fuerza Pinot Noir one has great red fruit aromas". Tina Fuerza Pinot Noir review titled "Great drop, great value" says: "Holds its on among the others but for less the price. Worth the try at $10 a bottle".', metadata={'Grape Varietal': 'Pinot Noir', 'Pairs Well With or Best Matched': 'Not specified', 'Price': 10.0, 'Region': 'Not specified', 'Vineyard': 'Tina Fuerza', 'Vintage Year': 2021.0, 'Wine Name': 'Tina Fuerza Pinot Noir'}), Document(page_content='Yellowglen White wine is not preservative free. Yellowglen White wine is not gluten free. Yellowglen White wine is not kosher. Yellowglen White brand of wine is Yellowglen. Yellowglen White product number is 362372. Yellowglen White wine is not organic. Yell

  suggest me wines whose price is $50?


Filter: query=' ' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='Price', value=50) limit=None
Retriever:  [Document(page_content='closure type is Stopper. Gerard Bertrand Cote des Roses rosé magnum 1.5L winery is rated 14.8 (2015). Gerard Bertrand Cote des Roses rosé magnum 1.5L food that is best matched with this wine is Oysters. Gerard Bertrand Cote des Roses rosé magnum 1.5L brand of wine is Gerard Bertrand. Gerard Bertrand Cote des Roses rosé magnum 1.5L product number is 501114.', metadata={'Grape Varietal': 'Rosé', 'Pairs Well With or Best Matched': 'Oysters', 'Price': 50.0, 'Region': 'Languedoc-Roussillon', 'Vineyard': 'Gerard Bertrand', 'Vintage Year': 2020.0, 'Wine Name': 'Gerard Bertrand Cote des Roses rosé magnum 1.5L'}), Document(page_content='Martinborough Vineyard Chardonnay is priced at $50 per bottle. Martinborough Vineyard Chardonnay grape varietal is Chardonnay. Martinborough Vineyard Chardonnay region of origin is Martinborough. Martinborough Vineyard

  suggest me wines whose price is $19?


Filter: query=' ' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='Price', value=19) limit=None
Retriever:  [Document(page_content='European wines because I find them too sweet. La Poco Spanish Tempranillo is a superb Spanish wine. It’s not sweet, it’s not bitter, it’s not dry, its perfect, it’s great on its own or with a simple spag bog! For the price of this wine you won’t be disappointed. I’ll buy it again.". La Poco Spanish Tempranillo review titled "By far a favourite" says: "Do yourself a huge favour and experience this one. It won\'t Disappoint".', metadata={'Grape Varietal': 'Not specified', 'Pairs Well With or Best Matched': 'Not specified', 'Price': 19.0, 'Region': 'Not specified', 'Vineyard': 'La Poco', 'Vintage Year': 2019.0, 'Wine Name': 'La Poco Spanish Tempranillo'}), Document(page_content='Xanadu Fusion Sauvignon Blanc is priced at $19 per bottle. Xanadu Fusion Sauvignon Blanc grape varietal is Sauvignon Blanc. Xanadu Fusion Sauvignon Blanc liquor style is